In [ ]:
import asyncio
from playwright.async_api import async_playwright
import aiohttp
import os
import nest_asyncio

nest_asyncio.apply()

async def download_image(session, url, filename):
    async with session.get(url) as resp:
        if resp.status == 200:
            with open(filename, 'wb') as f:
                f.write(await resp.read())

async def crawl_tour_detail(context, detail_url):
    page = await context.new_page()
    await page.goto(detail_url)
    await page.wait_for_timeout(3000)

    try:
        itinerary_buttons = await page.locator("button.accordion-button").all()
        itinerary = "\n".join([await btn.inner_text() for btn in itinerary_buttons])
        

    except:
        itinerary = "Không có dữ liệu lịch trình"

    try:
        includes = await page.locator("div.includes").inner_text()
    except:
        includes = "Không rõ dịch vụ bao gồm"

    await page.close()
    return {
        "itinerary": itinerary,
        "includes": includes
    }

async def crawl_tours():
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        context = await browser.new_context()
        page = await context.new_page()
        base_url = "https://hanoitourist.vn/tour-trong-nuoc"
        await page.goto(base_url)
        await page.wait_for_timeout(5000)

        current_page = 1
        max_page = 1  # Đổi số trang muốn crawl tại đây

        all_tours = []
        async with aiohttp.ClientSession() as session:
            while current_page <= max_page:
                print(f"🔎 Crawling page {current_page}...")

                tour_cards = await page.locator("div.tour-item").all()

                for i, card in enumerate(tour_cards):
                    try:
                        title = await card.locator("h3 a").inner_text()
                        price = await card.locator("div.detail-gia").inner_text()
                        location = await card.locator("div.detail-item-value >> nth=1").inner_text()
                        dates = await card.locator("div.detail-item-value >> nth=0").inner_text()
                        detail_url = await card.locator("h3 a").get_attribute("href")
                        full_detail_url = f"https://hanoitourist.vn{detail_url}" if detail_url.startswith("/") else detail_url
                        print(full_detail_url)
                        image_url = await card.locator("div.tour-img img").get_attribute("src")
                        filename = f"tour_image_{current_page}_{i+1}.jpg"

                        # if image_url:
                        #     await download_image(session, image_url, filename)

                        # Crawl chi tiết từng tour
                        detail_data = await crawl_tour_detail(context, full_detail_url)

                        tour_info = {
                            "title": title,
                            "location": location,
                            "dates": dates,
                            "price": price,
                            "image_url": image_url,
                            "detail_url": full_detail_url,
                            **detail_data
                        }

                        print(f"✅ {title}")
                        print(f"📍 {location} | 📅 {dates} | 💰 {price}")
                        print(f"📝 Lịch trình: {detail_data['itinerary'][:100]}...")
                        print("────────────")

                        all_tours.append(tour_info)
                    except Exception as e:
                        print("⚠️ Lỗi xử lý card:", e)

                # Tìm nút "Trang tiếp" để next page
                try:
                    next_btn = page.locator("a.page-link:has-text('>')")
                    if await next_btn.is_visible():
                        await next_btn.click()
                        await page.wait_for_timeout(3000)
                        current_page += 1
                    else:
                        break
                except:
                    break

        await browser.close()

        print(f"🎉 Tổng cộng thu được {len(all_tours)} tour.")
        
        # Có thể lưu `all_tours` vào file CSV hoặc JSON nếu muốn.

# Chạy
await crawl_tours()


🔎 Crawling page 1...
https://hanoitourist.vn/can-tho-soc-trang-con-dao
✅ CẦN THƠ - SÓC TRĂNG - CÔN ĐẢO
📍 Ngày đi: 19/04, 17/05, 20/06/2025... (4 ngày 3 đêm) | 📅 Hành trình: TP. Hồ Chí Minh - Cần Thơ - Côn Đảo - 4 ngày 3 đêm | 💰 8,790,000đ
📝 Lịch trình: NGÀY 01: HÀ NỘI - CẦN THƠ - SÓC TRĂNG (ĂN TỐI)
NGÀY 02: SÓC TRĂNG - CÔN ĐẢO (ĂN TRƯA, TỐI)
NGÀY 03: ...
────────────
https://hanoitourist.vn/can-tho-cao-lanh-chau-doc-giang-can-tho
✅ CẦN THƠ - CAO LÃNH - CHÂU ĐỐC - AN GIANG - CẦN THƠ
📍 Ngày đi: 19/04, 17/05, 20/06/2025... (4 ngày 3 đêm) | 📅 Hành trình: Cần Thơ - TP. Hồ Chí Minh - 4 ngày 3 đêm | 💰 8,150,000đ
📝 Lịch trình: NGÀY 1: HÀ NỘI - CẦN THƠ - CAO LÃNH (Ăn tối)
NGÀY 2: CAO LÃNH - TRI TÔN - RỪNG TRÀM - CHÂU ĐỐC (Ăn s...
────────────
https://hanoitourist.vn/vinwonder-bai-tranh-lang-chai-vinh-san-ho-dong-cuu-hang-rai-vuon-nho-ninh-thuan-thap-ba-ponagar-tam
✅ VINWONDER – BÃI TRANH – LÀNG CHÀI – VỊNH SAN HÔ ĐỒNG CỪU – HANG RÁI - VƯỜN NHO NINH THUẬN THÁP BÀ PONAGAR – TẮM BÙN I’RESORT
📍 Ngà

In [6]:
async def crawl_tripadvisor_carousel_images_updated(url, num_clicks=10):
    from playwright.async_api import async_playwright
    from bs4 import BeautifulSoup
    import re
    from urllib.parse import urljoin

    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=False)  # Debug bằng browser thật
        page = await browser.new_page()
        await page.goto(url)
        await page.wait_for_timeout(3000)

        # Scroll để hiện carousel
        await page.evaluate("window.scrollBy(0, 1000)")
        await page.wait_for_timeout(2000)

        # Click mũi tên phải nhiều lần
        for i in range(num_clicks):
            try:
                next_btn = page.locator('button.pWJww')
                if await next_btn.is_visible():
                    await next_btn.click()
                    await page.wait_for_timeout(800)
                else:
                    print(f"[{i}] Không thấy nút next.")
            except Exception as e:
                print(f"[{i}] ❌ Không bấm được: {e}")
                break

        html = await page.content()
        soup = BeautifulSoup(html, "html.parser")

        image_urls = set()

        # ✅ Quét tất cả div chứa ảnh: ZGLUM là container
        for container in soup.select("div.ZGLUM"):
            for img in container.select("img"):
                src = img.get("src") or img.get("data-src") or ""
                if not src:
                    continue
                src = re.sub(r"-s\d+x\d+", "-s1600x1200", src).split("?")[0]
                if not src.startswith("http"):
                    src = urljoin("https://www.tripadvisor.com", src)
                if "icons" in src.lower():
                    continue
                image_urls.add(src)

            # ✅ Nếu có thẻ <source> (ảnh responsive)
            for source in container.select("source"):
                srcset = source.get("srcset", "")
                for part in srcset.split(","):
                    candidate = part.strip().split(" ")[0]
                    candidate = re.sub(r"-s\d+x\d+", "-s1600x1200", candidate).split("?")[0]
                    if candidate.startswith("http"):
                        image_urls.add(candidate)

        await browser.close()
        return list(image_urls)


In [8]:
import nest_asyncio
nest_asyncio.apply()

url = "https://www.tripadvisor.com/Attraction_Review-g293924-d317503-Reviews-Old_Quarter-Hanoi.html"
imgs = await crawl_tripadvisor_carousel_images_updated(url, num_clicks=20)

print(f"🎉 Tìm được {len(imgs)} ảnh:")
for img in imgs[:10]:
    print(img)


[0] Không thấy nút next.
[1] Không thấy nút next.
[2] Không thấy nút next.
[3] Không thấy nút next.
[4] Không thấy nút next.
[5] Không thấy nút next.
[6] Không thấy nút next.
[7] Không thấy nút next.
[8] Không thấy nút next.
[9] Không thấy nút next.
[10] Không thấy nút next.
[11] Không thấy nút next.
[12] Không thấy nút next.
[13] Không thấy nút next.
[14] Không thấy nút next.
[15] Không thấy nút next.
[16] Không thấy nút next.
[17] Không thấy nút next.
[18] Không thấy nút next.
[19] Không thấy nút next.
🎉 Tìm được 4 ảnh:
https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2f/9b/57/24/caption.jpg
https://dynamic-media-cdn.tripadvisor.com/media/photo-o/09/32/1d/c4/old-quarter.jpg
https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2f/9c/23/45/caption.jpg
https://dynamic-media-cdn.tripadvisor.com/media/photo-o/2f/9b/57/23/caption.jpg


In [1]:
# crawl_foody_hanoi.py

from playwright.sync_api import sync_playwright
import pandas as pd
import time

def crawl_foody_hanoi(pages_to_crawl=3):
    results = []

    with sync_playwright() as p:
        browser = p.chromium.launch(headless=True)
        page = browser.new_page()
        base_url = "https://www.foody.vn/ha-noi/dia-diem?page="

        for i in range(1, pages_to_crawl + 1):
            print(f"Crawling page {i}...")
            page.goto(base_url + str(i), timeout=60000)
            page.wait_for_timeout(5000)

            items = page.query_selector_all("div.result-group > div[data-view-id]")
            for item in items:
                try:
                    name = item.query_selector("h2 > a").inner_text().strip()
                    url = item.query_selector("h2 > a").get_attribute("href")
                    full_url = "https://www.foody.vn" + url

                    address = item.query_selector(".result-address").inner_text().strip()
                    rating = item.query_selector(".point.highlight").inner_text().strip() if item.query_selector(".point.highlight") else "N/A"
                    category = item.query_selector(".detail-info").inner_text().strip() if item.query_selector(".detail-info") else "N/A"
                    img_url = item.query_selector("img").get_attribute("src")

                    results.append({
                        "Tên": name,
                        "Địa chỉ": address,
                        "Điểm": rating,
                        "Loại hình": category,
                        "Link ảnh": img_url,
                        "Foody URL": full_url
                    })
                except Exception as e:
                    print(f"❌ Error on item: {e}")
                    continue

        browser.close()

    return results

if __name__ == "__main__":
    data = crawl_foody_hanoi(pages_to_crawl=5)
    df = pd.DataFrame(data)
    df.to_csv("foody_hanoi.csv", index=False, encoding="utf-8-sig")
    print("✅ Đã lưu kết quả vào foody_hanoi.csv")


Error: It looks like you are using Playwright Sync API inside the asyncio loop.
Please use the Async API instead.